In [2]:
import tensorflow as tf
# 手写数字相关工具包
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# 载入数据集
# 没有的话会自动去下载数据集
# 数据路径  转为one_hot(某一位数字为1，其余数字都为0)格式
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
# 每个批次的大小
'''
优化
    可修改批次大小
    可添加隐藏层
'''
# 每次放入批次大小的数据集
# 形式为矩阵
batch_size = 100
# 批次的个数
# 计算一共有多少个批次
# 总训练集  整除  批次大小
n_batch = mnist.train.num_examples // batch_size

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [4]:
'''
可视化命名空间
'''
with tf.name_scope('input'):
    # 输入
    # 定义两个placeholder
    # [行 -> 任意值(此处为100，与上面的批次大小一致), 列 -> 每张图片都是28*28，需要转为一维的向量也就是28*28=784]
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    # 数字为 0-9 
    y = tf.placeholder(tf.float32, [None, 10], name='y_input')

with tf.name_scope('layer'):
    # 输出
    # 创建简单的神经网络
    '''
    可修改初始值 ?
    '''
    with tf.name_scope('weights'):
        # 权值 -> 当前权值 784个输入层， 10个输出层
        W = tf.Variable(tf.zeros([784,10]), name='W')
    with tf.name_scope('biases'):
        # 偏置值
        b = tf.Variable(tf.zeros([10]), name='b')
    # 信号和    
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x, W) + b
    # 预测值
    # 激活函数 -> 需用softmax函数
    # 数据和权值矩阵相乘 + 偏置值 再 使用softmax函数激活
    # softmax -> 转换为概率值
    with tf.name_scope('softmax'):
        # 预测值
        prediction = tf.nn.softmax(wx_plus_b)

# 优化
'''
可选择优化方式，比如交叉熵
'''
    # 二次代价函数
    # 误差值
    # 真实值 - 预测值 的 平方 的 平均值
    # loss = tf.reduce_mean(tf.square(y - prediction))
with tf.name_scope('loss'):
    # softmax交叉熵代价函数
    # params => 标签值(真实值)， 预测值   
    # reduce_mean -> 平均值
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
# 梯度下降法优化 loss
''' 
学习率可修改
'''
with tf.name_scope('train'):
    # 0.2的学习率最小化loss
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 测试准确率 => 存放布尔值的列表
        # tf.equal -> 比较参数一(真实值数据)和参数二(预测值数据)行[或者列]的最大值 => True or False
        # argmax -> 返回一维张量中最大值所在的位置
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
    with tf.name_scope('accuracy'):
        # 求准确率
        # tf.cast -> 将对比后的布尔值列表转换为对应的浮点值 => True为1.0，False为0
        # tf.reduce_mean -> 平均值
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    # 先初始化变量
    sess.run(tf.global_variables_initializer())
    '''
    画 params => 路径, 图结构(graph)
    '''
    writer = tf.summary.FileWriter('logs/', sess.graph)
    # 21个周期
    # 图片训练 21 个周期
    for epoch in range(1):
        # 批次
        # 所有的图片都训练一次
        for batch in range(n_batch):
            # batch_xs
            # 获得一个批次，每次大小为100
            # 相当于每次获取100张图片
            # batch_ys
            # 图片的标签
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 执行训练传入数据
            sess.run(train_step,feed_dict={x:batch_xs, y:batch_ys})
        # 每个周期的准确率
        # 测试集的图片和测试集的图片标签
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print('(Iter)第\t%s\t个周期(Testing Accuracy)准确率\t%s' % (str(epoch), str(acc)))

(Iter)第	0	个周期(Testing Accuracy)准确率	0.8243
